# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Trend = candidate_subclass('Trend', ['tr'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Trend).filter(Trend.split == 0).order_by(Trend.id).all()
dev_cands   = session.query(Trend).filter(Trend.split == 1).order_by(Trend.id).all()
test_cands  = session.query(Trend).filter(Trend.split == 2).order_by(Trend.id).all()
# tmp_cands = session.query(Trend).filter(Trend.split == 3).order_by(Trend.id).all()
# print(tmp_cands)

Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [6]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=103  #epochs=10  batch size=64
[LSTM] Epoch 1 (0.15s)	Average loss=0.693292	Dev F1=0.00


/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[LSTM] Epoch 2 (0.29s)	Average loss=0.675111	Dev F1=0.00
[LSTM] Epoch 3 (0.42s)	Average loss=0.670821	Dev F1=0.00
[LSTM] Epoch 4 (0.57s)	Average loss=0.668241	Dev F1=0.00
[LSTM] Epoch 5 (0.71s)	Average loss=0.667019	Dev F1=0.00
[LSTM] Epoch 6 (0.88s)	Average loss=0.664243	Dev F1=0.00
[LSTM] Epoch 7 (1.00s)	Average loss=0.659737	Dev F1=0.00
[LSTM] Epoch 8 (1.14s)	Average loss=0.658620	Dev F1=0.00
[LSTM] Epoch 9 (1.27s)	Average loss=0.655183	Dev F1=0.00
[LSTM] Epoch 10 (1.41s)	Average loss=0.656320	Dev F1=0.00
[LSTM] Training done (1.44s)


Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.000, Recall: 0.000, F1 Score: 0.000


/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
from snorkel.viewer import SentenceNgramViewer
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)
sv = SentenceNgramViewer(fp, session)
sv

Scores (Un-adjusted)
Pos. class accuracy: 0.0
Neg. class accuracy: 1.0
Precision            0.0
Recall               0.0
F1                   0.0
----------------------------------------
TP: 0 | FP: 0 | TN: 14 | FN: 3



/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


<IPython.core.display.Javascript object>

SentenceNgramViewer(html='<head>\n<style>\nspan.candidate {\n    background-color: rgba(255,255,0,0.3);\n}\n\n…

Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [9]:
#lstm.save_marginals(session, test_cands)
# load and save the unseen cands here
from snorkel.viewer import SentenceNgramViewer
#sv = SentenceNgramViewer(fn, session)
#sv
s = lstm.marginals(tmp_cands)
# s2 = lstm.predictions(tmp_cands)
# sv = SentenceNgramViewer(, session)
# sv
# lstm.save_marginals(session, tmp_cands)
#lstm.save_marginals(session, unseen_cands)
# use the marginals here and extract 
# tmp_cands = session.query(Trend).filter(Trend.split == 3).order_by(Trend.id).all()
# print(len(tmp_cands))
# print(lstm.predictions(tmp_cands))
sv = SentenceNgramViewer(tn, session)
sv

NameError: name 'tmp_cands' is not defined

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!